In [7]:
import sys
from google.colab import drive
from pathlib import Path
!gdown --id 1T1lbyjEYdI9pLX2f-hFzX0q7a5TETqH1

Downloading...
From: https://drive.google.com/uc?id=1T1lbyjEYdI9pLX2f-hFzX0q7a5TETqH1
To: /content/Final_Combained_Data_Final.xlsx
100% 105k/105k [00:00<00:00, 1.63MB/s]


In [36]:
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [8]:
#needed packages to install
#!pip install texthero==1.0.5
!pip install gensim
!pip install openpyxl 
!pip install bnlp_toolkit
!pip install python-bidi
!pip install texthero
!pip install bangla-stemmer

  Using cached gensim-4.0.1-cp37-cp37m-manylinux1_x86_64.whl (23.9 MB)
  Attempting uninstall: gensim
    Found existing installation: gensim 3.8.3
    Uninstalling gensim-3.8.3:
      Successfully uninstalled gensim-3.8.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
texthero 1.1.0 requires gensim<4.0,>=3.6.0, but you have gensim 4.0.1 which is incompatible.


  Using cached gensim-3.8.3-cp37-cp37m-manylinux1_x86_64.whl (24.2 MB)
  Attempting uninstall: gensim
    Found existing installation: gensim 4.0.1
    Uninstalling gensim-4.0.1:
      Successfully uninstalled gensim-4.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bnlp-toolkit 3.1.2 requires gensim==4.0.1, but you have gensim 3.8.3 which is incompatible.


In [9]:
#import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
from wordcloud import WordCloud
from bnlp.corpus import stopwords, punctuations
import bnlp
from wordcloud import WordCloud, STOPWORDS , ImageColorGenerator
from bnlp import BasicTokenizer,NLTKTokenizer
from bangla_stemmer.stemmer.stemmer import BanglaStemmer
import warnings
warnings.filterwarnings("ignore")

In [11]:
df = pd.read_excel('/content/Final_Combained_Data_Final.xlsx')


df_plot=df.copy()
df.head()

,Speech,Label
0,"ওনাদের খাবারের পরিমাণ দেখে আমরা পুরাই অবাক, 55...",0.0
1,করুণ খাবারের মান। বেশি দাম. এটা এতটাই খারাপ যে...,0.0
2,"বাজে ছিল , খাবার গরম করে দিলে হয়তবা ভাল লাগত ।",0.0
3,"তবে সমস্যা যেটা হয়েছে, স্টেক এতই হার্ড ছিলো যে...",0.0
4,তেহেরি খুব একটা ভালো লাগে নাই।,0.0


In [12]:
from sklearn.utils import shuffle
df = shuffle(df)
df=df.reset_index(drop=True)
df.head()

,Speech,Label
0,পায়েস আর একটু বাড়ালে আরো ভালো হতো,2.0
1,ফ্রেশ খাবার ছিল দুপুরে খেয়ে খুব মজা পেয়েছি,1.0
2,ঝোলে মাখা ঝরঝরে খিচুড়ি আর ব্রেডক্রাম্বে মোড়ানো...,1.0
3,কালা ভূনা খেতে মুটামুটি,2.0
4,ফুসকা চুপসানো ছিল,0.0


In [13]:
df_copy=df.copy()

In [14]:
def demoji(text):
	emoji_pattern = re.compile("["
		u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U00010000-\U0010ffff"
	                           "]+", flags=re.UNICODE)
	return(emoji_pattern.sub(r'', text)) 


def clean(text):
    text = re.sub('[%s]' % re.escape(punctuations), ' ', text)     #escape punctuation
    text = re.sub('\n', ' ', text)                                 #replace line break with space
    text = re.sub('\w*\d\w*', ' ', text)                           #ignore digits
    #text = re.sub('\xa0', ' ', text)                              
    return text

def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[\u09E6-\u09FF]+', ' ', text)                  #remove bangla punctuations
    return text

In [16]:
df[u'text'] = df[u'Speech'].astype(str)
df[u'text'] = df[u'text'].apply(lambda x:demoji(x))
df['text'] = df['text'].apply(lambda x: re.split('http:\/\/.*', str(x))) #remove urls
df["text"] = df['text'].apply(lambda x: clean(str(x)))                      
df['text'] = df['text'].apply(lambda x: remove_punct(x))

In [17]:
#remove special characters
spec_chars = ["!",'"',"।","#","%","&","'","(",")",
              "*","+",",","-",".","/",":",";","<",
              "=",">","?","@","[","\\","]","^","_",
              "`","{","|","}","~","–"]
for char in spec_chars:
    df['text'] = df['text'].str.replace(char, ' ') 
    df['text'] = df['text'].str.split().str.join(' ')         #remove whitespace

In [18]:
df.text[5]

'রোস্ট এভারেজ লেগেছ'

In [19]:
custom_stop_word_list=['আমার ','অথচ ','অথবা ','অনুযায়ী ','অনেক ','অনেকে ','অনেকেই ','অন্তত ','অন্য ','অবধি ','অবশ্য ','অর্থাত ','আই ','আগামী ','আগে ','আগেই ','আছে ','আজ ','আদ্যভাগে ',
                       'আপনার ','আপনারা ','আপনি ','আবার ','আসবে ','আমরা ',' আমাকে ','আমাদের ','আমার ','আমি ','আর ','আরও ','ইত্যাদি ','ইহা ','উচিত ','উত্তর ','উনি ','উপর ','উপরে ','এ ','এঁদের ','এঁরা ','এরা ',
                       'এই ','একই ','একটি ','একবার ','একে ','এক্ ','এখন ','এখনও ','এখানে ','এখানেই ','এটা ','এটাই ','এটি ','এত ','এতটাই ','এতে ','এদের ','এব ','এবং ','এবার ','এমন ','এমনকী ',
                       'এমনি ','এর ','এরা ','এল ','এস ','এসে ','ঐ ','ওঁদের ','ওঁর ','ওঁরা ','ওই ','ওকে ','ওখানে ','ওদের ','ওর ','ওরা ','কখনও ','কত ','কবে ','কমনে ','কয়েক ','কয়েকটি ','করছে ',
                       'করছেন ','করতে ',' করবে',' করবেন',' করলে ',' করলেন',' করা',' করাই',' করায়',' করার',' করি','করতে ','করিতে ','করিয়া ','করিয়ে ','করে ','করেই ','করেছিলেন ','করেছে ','করেছেন ','করেন ',
                       'কাউকে ','কাছ ','কাছে ','কাজ ','কাজে ','কারও ','কারণ ','কি ','কিংবা ','কিছু ','কিছুই ','হেতি ','কিন্তু ','ন্তু ','কী ','কে ','কেউ ','কেউই ','কেখা ','কেন ','কোটি ','কোন ','কোনও ',
                       'কোনো ','ক্ষেত্রে ','কয়েক ','খুব ','গিয়ে ','গিয়েছে ','গেছেন ','গিয়ে ','গুলি ','গেছে ','গেল ','গেলে ','গোটা ','চলে ','চান ','চায় ','চার ','চালু ','চেয়ে ','চেষ্টা ','ছাড়া ','ছাড়াও ','ছিল ','ছিলেন ','জন ',
                       'জনকে ','জনের ','জন্য ','জন্যওজে ','জানতে ','জানা ','জানানো ','জানায় ','জানিয়ে ','জানিয়েছে ','জ্নজন ','জন ','টা ','টি ','ঠিক ','তখন ','তত ','তথা ','তবু ','তবে ','তা ','তাঁকে ','তাঁদের ',
                       'তাঁর ','তোর ','তাঁরা ','তাঁহারা ','তাই ','যে ''তাও ','তাকে ','তাতে ','তাদের ','তার ','তারপর ','তারা ','তারৈ ','তাহলে ','তাহা ','তাহাতে ' ,'তাহার ','তিনঐ ','তিনি ','তিনিও ','তুমি ','তুলে ','তেমন ','তো ','তোমার ',
                       'থাকবে ','থাকবেন ','থাকা ','থাকায় ','থাকে ','থাকেন ','থেকে ','থেকেই ','থেকেও ','দিকে ','দিতে ','দিতাম','দিন ','দিয়ে ','দিয়েছে ','দিয়েছেন ','দিলেন ', 'দু ','দুই ','দুটি ','দুটো ','দেওয়া ','দেওয়ার ','দেওয়া ',
                       'দেখতে ','দেখা ','দেখে ','দেন ','দেয়া ','দেয় ','দ্বারা ','ধরা ','ধরে ','ধামার ','নতুন ','নাই ','নাকি ','নাগাদ ','নানা ','নিজে ','নিজেই ','নিজেদের ','নিজের ','নিতে ','নিয়ে ','নিয়ে ','নেই ','নেওয়া ','নেওয়ার ',
                       'নেওয়া ','নয় ','পক্ষে ','পর ','পরে ','পরেই ','পরেও ','পর্যন্ত ','পাওয়া ','পাচ ','পারি ','পারে ','পারেন ','পেয়ে ','পেয়্র্ ','প্রতি ','প্রথম ','প্রভৃতি ','প্রযন্ত ','প্রাথমিক ','প্রায় ','প্রায় ','ফলে ','ফিরে ','ফের ',
                       'বক্তব্য ','বদলে ','বন ','বরং ','বলতে ','বলছি ','বলল ','বললেন ','বলা ','বলে ','বলেছেন ','বলেন ','বসে ','বহু' ,'বাদে ','বার ','বিনা ','বিভিন্ন ','বিশেষ ','বিষয়টি ','বেশ ','বেশি ','ব্যবহার ','ব্যাপারে ','ভাবে ', 'ভাবেই ',
                       'মতো ','মতোই ','মধ্যভাগে ','মধ্যে ','মধ্যেই ','মধ্যেও ','মনে ','মাত্র ','মাধ্যমে ','মোট ','মোটেই ','যখন ','যত ','যতটা ','যথেষ্ট ','যদি ','যদিও ','যা ','যাঁর ','যাঁরা ','যাওয়া ','যাওয়ার ','যাওয়া ','যাকে ','যাচ্ছে ',
                       'যাতে ','যাদের ','যান ','যাবে ','যায় ','যার ','যারা ','যিনি ','অতএব ','যেখানে ','যেতে ','যেন ','যেমন ','রকম ','রয়েছে ','রাখা ','রেখে ','লক্ষ ','শুধু ','শুরু ','সঙ্গে ','সঙ্গেও ','সব ','সবার ','সবাইর ','সমস্ত ',
                       'সম্প্রতি ','সহ ','সহিত ','সবই ','সাধারণ ','সামনে ','সুতরাং ','সবাইর ','সে ','সেই ','সেখান ','সেখানে ','সেটা ', 'সেটাই ','সেটাও ','সেটি ','স্পষ্ট ','স্বয়ং ','হইতে ','হইবে ','হইয়া ','হওয়া ','হওয়ায় ','হওয়ার ','হচ্ছে ','হত ','হতে ',
                       'লেগেছে ','হতেই ','হন ','হইত ','হবে ','তিনি ','হবেন ','হয় ','হয়তো ','হয়নি ','হয়ে ','হয়েই ','হয়েছিল ','হয়েছে ','হয়েছেন ','হল ','হলে ','হলেই ','হলেও ','হলো ','হাজার ','হিসাবে ','হৈলে ','হোক ','হয় ']

#digits=['০ ','১ ','২ ','৩ ','৪ ','৫ ','৬ ','৭ ','৮ ','৯ ']

final_stopword_list = custom_stop_word_list 

pat = r'\b(?:{})\b'.format('|'.join(final_stopword_list))
df['text'] = df['text'].str.replace(pat, ' ')
df['text'] = df['text'].str.replace(r'\s+', ' ')

In [20]:
df["clean_text"]=df["text"]

In [21]:
def stemming_text(corpus):
    stm = BanglaStemmer()
    return [' '.join([stm.stem(word) for word in review.split()]) for review in corpus]

df['text'] = stemming_text(df['text'])


Streaming output truncated to the last 5000 lines.
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied second rules..
applied third rules..
applied second rules..
applied second rules..
applied second rules..
applied second rules..
applied first rules..
applied fourth rules..
applied third rules..
applied first rules..
applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied first rules..
applied third rules..
applied second rules..
applied second rules..
applied fourth rules..
applied second rules..
applied first rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..


In [22]:
df.head()

,Speech,Label,text,clean_text
0,পায়েস আর একটু বাড়ালে আরো ভালো হতো,2.0,পায়েস একটু বাড়ালে আরো ভালো হত,পায়েস একটু বাড়ালে আরো ভালো হতো
1,ফ্রেশ খাবার ছিল দুপুরে খেয়ে খুব মজা পেয়েছি,1.0,ফ্রেশ খা দুপুরে খেয়ে মজা পেয়ে,ফ্রেশ খা দুপুরে খেয়ে মজা পেয়েছি
2,ঝোলে মাখা ঝরঝরে খিচুড়ি আর ব্রেডক্রাম্বে মোড়ানো...,1.0,ঝোলে মাখা ঝরঝরে খিচুড়ি ব্রেডক্রাম্বে মোড়ানো আল...,ঝোলে মাখা ঝরঝরে খিচুড়ি ব্রেডক্রাম্বে মোড়ানো আল...
3,কালা ভূনা খেতে মুটামুটি,2.0,কালা ভূনা খেত মুটামু,কালা ভূনা খেতে মুটামুটি
4,ফুসকা চুপসানো ছিল,0.0,ফুসকা চুপসানো ছিল,ফুসকা চুপসানো ছিল


In [23]:
#Tokenization

b_token = BasicTokenizer()
df['tokenized_clean_text'] = df.apply(lambda row: b_token.tokenize(row['clean_text']), axis=1)
df['tokenized_stem_text'] = df.apply(lambda row: b_token.tokenize(row['text']), axis=1)

df['token_length'] = df.apply(lambda row: len(row['tokenized_clean_text']), axis=1)

df.head() 

,Speech,Label,text,clean_text,tokenized_clean_text,tokenized_stem_text,token_length
0,পায়েস আর একটু বাড়ালে আরো ভালো হতো,2.0,পায়েস একটু বাড়ালে আরো ভালো হত,পায়েস একটু বাড়ালে আরো ভালো হতো,"[পায়েস, একটু, বাড়ালে, আরো, ভালো, হতো]","[পায়েস, একটু, বাড়ালে, আরো, ভালো, হত]",6
1,ফ্রেশ খাবার ছিল দুপুরে খেয়ে খুব মজা পেয়েছি,1.0,ফ্রেশ খা দুপুরে খেয়ে মজা পেয়ে,ফ্রেশ খা দুপুরে খেয়ে মজা পেয়েছি,"[ফ্রেশ, খা, দুপুরে, খেয়ে, মজা, পেয়েছি]","[ফ্রেশ, খা, দুপুরে, খেয়ে, মজা, পেয়ে]",6
2,ঝোলে মাখা ঝরঝরে খিচুড়ি আর ব্রেডক্রাম্বে মোড়ানো...,1.0,ঝোলে মাখা ঝরঝরে খিচুড়ি ব্রেডক্রাম্বে মোড়ানো আল...,ঝোলে মাখা ঝরঝরে খিচুড়ি ব্রেডক্রাম্বে মোড়ানো আল...,"[ঝোলে, মাখা, ঝরঝরে, খিচুড়ি, ব্রেডক্রাম্বে, মোড়...","[ঝোলে, মাখা, ঝরঝরে, খিচুড়ি, ব্রেডক্রাম্বে, মোড়...",13
3,কালা ভূনা খেতে মুটামুটি,2.0,কালা ভূনা খেত মুটামু,কালা ভূনা খেতে মুটামুটি,"[কালা, ভূনা, খেতে, মুটামুটি]","[কালা, ভূনা, খেত, মুটামু]",4
4,ফুসকা চুপসানো ছিল,0.0,ফুসকা চুপসানো ছিল,ফুসকা চুপসানো ছিল,"[ফুসকা, চুপসানো, ছিল]","[ফুসকা, চুপসানো, ছিল]",3


In [24]:
df.describe()

,Label,token_length
count,2187.000000,2187.000000
mean,1.010517,8.294467
std,0.818015,8.577358
min,0.000000,0.000000
25%,0.000000,3.000000
50%,1.000000,5.000000
75%,2.000000,10.000000
max,2.000000,103.000000


In [25]:
df.head()

,Speech,Label,text,clean_text,tokenized_clean_text,tokenized_stem_text,token_length
0,পায়েস আর একটু বাড়ালে আরো ভালো হতো,2.0,পায়েস একটু বাড়ালে আরো ভালো হত,পায়েস একটু বাড়ালে আরো ভালো হতো,"[পায়েস, একটু, বাড়ালে, আরো, ভালো, হতো]","[পায়েস, একটু, বাড়ালে, আরো, ভালো, হত]",6
1,ফ্রেশ খাবার ছিল দুপুরে খেয়ে খুব মজা পেয়েছি,1.0,ফ্রেশ খা দুপুরে খেয়ে মজা পেয়ে,ফ্রেশ খা দুপুরে খেয়ে মজা পেয়েছি,"[ফ্রেশ, খা, দুপুরে, খেয়ে, মজা, পেয়েছি]","[ফ্রেশ, খা, দুপুরে, খেয়ে, মজা, পেয়ে]",6
2,ঝোলে মাখা ঝরঝরে খিচুড়ি আর ব্রেডক্রাম্বে মোড়ানো...,1.0,ঝোলে মাখা ঝরঝরে খিচুড়ি ব্রেডক্রাম্বে মোড়ানো আল...,ঝোলে মাখা ঝরঝরে খিচুড়ি ব্রেডক্রাম্বে মোড়ানো আল...,"[ঝোলে, মাখা, ঝরঝরে, খিচুড়ি, ব্রেডক্রাম্বে, মোড়...","[ঝোলে, মাখা, ঝরঝরে, খিচুড়ি, ব্রেডক্রাম্বে, মোড়...",13
3,কালা ভূনা খেতে মুটামুটি,2.0,কালা ভূনা খেত মুটামু,কালা ভূনা খেতে মুটামুটি,"[কালা, ভূনা, খেতে, মুটামুটি]","[কালা, ভূনা, খেত, মুটামু]",4
4,ফুসকা চুপসানো ছিল,0.0,ফুসকা চুপসানো ছিল,ফুসকা চুপসানো ছিল,"[ফুসকা, চুপসানো, ছিল]","[ফুসকা, চুপসানো, ছিল]",3


In [38]:
df1=df.copy()


df.to_csv(encoding='utf-8', index=False)
df.to_csv('/content/drive/MyDrive/Colab Notebooks/comments_tokenized.csv', index = False, header=True)
df.tail()

,Label,text,clean_text,tokenized_clean_text,tokenized_stem_text,token_length
2182,0.0,বোরহানি বাসি পানি মিশানো ছিল,বোরহানি বাসি পানি মিশানো ছিল,"[বোরহানি, বাসি, পানি, মিশানো, ছিল]","[বোরহানি, বাসি, পানি, মিশানো, ছিল]",5
2183,2.0,কালা ভূনা একটু জম,কালা ভূনা একটু জমতো,"[কালা, ভূনা, একটু, জমতো]","[কালা, ভূনা, একটু, জম]",4
2184,0.0,সত্যি স্বা সাথে পরিমাণ কমিয়ে স্বাদ আগ ভালো না...,সত্যিই স্বাদের সাথে পরিমাণও কমিয়ে স্বাদ আগের...,"[সত্যিই, স্বাদের, সাথে, পরিমাণও, কমিয়ে, স্বাদ...","[সত্যি, স্বা, সাথে, পরিমাণ, কমিয়ে, স্বাদ, আগ,...",12
2185,1.0,ভাত কম পেয়েছি স্বাদ ছিল,ভাত কম পেয়েছি স্বাদ ছিল,"[ভাত, কম, পেয়েছি, স্বাদ, ছিল]","[ভাত, কম, পেয়েছি, স্বাদ, ছিল]",5
2186,1.0,সের দাম সের জুস,সেরা দাম সেরা জুস,"[সেরা, দাম, সেরা, জুস]","[সের, দাম, সের, জুস]",4


In [39]:
from bnlp import POS
df.tokenized_clean_text = df.tokenized_clean_text.astype(str)
def pos_tagging(doc):
  bn_pos = POS()
  model_path = "/content/drive/MyDrive/Colab Notebooks/bn_pos.pkl"
  doc = bn_pos.tag(model_path,doc)
  return doc

In [40]:
def get_postags(row):
    postags = pos_tagging(row["tokenized_clean_text"])
    list_classes = list()
    for  word in postags:
        list_classes.append(word[1])
    return list_classes

df["postags_list"] = df.apply(lambda row: get_postags(row), axis = 1)
df['postags_list'] = df.postags_list.apply(lambda x: [i for i in x if i != 'PU'])
df['postags_list'] = df.postags_list.apply(lambda x: [i for i in x if i != 'RDS'])
df.sample(10, random_state = 4)

,Label,text,clean_text,tokenized_clean_text,tokenized_stem_text,token_length,postags_list
872,1.0,ভাল প্রাইজ হিসেব ভাল আসলে টেস্ট একেকজন একেকরকম...,ভালই প্রাইজ হিসেবে ভাল আসলে টেস্ট একেকজনের এক...,"['ভালই', 'প্রাইজ', 'হিসেবে', 'ভাল', 'আসলে', 'ট...","[ভাল, প্রাইজ, হিসেব, ভাল, আসলে, টেস্ট, একেকজন,...",10,"[NP, NP, PP, JJ, VM, NP, NP, JQ, NP, CX]"
157,1.0,মাংশ গন্ধ কেমন যেন,মাংশ গন্ধটা কেমন যেন,"['মাংশ', 'গন্ধটা', 'কেমন', 'যেন']","[মাংশ, গন্ধ, কেমন, যেন]",4,"[NC, NC, PWH, CSB]"
1079,2.0,ফ্রেন্স ফ্রা একটু বাড়ালে আরো ভালো হত,ফ্রেন্স ফ্রাই একটু বাড়ালে আরো ভালো হতো,"['ফ্রেন্স', 'ফ্রাই', 'একটু', 'বাড়ালে', 'আরো', ...","[ফ্রেন্স, ফ্রা, একটু, বাড়ালে, আরো, ভালো, হত]",7,"[NP, NP, JQ, NC, JQ, JJ, VM]"
1094,0.0,গতকাল খা ভালো লাগছিলো অর্ড করি গতকাল তুলনায় ভা...,গতকালের খা ভালোই লাগছিলো অর্ডার করি গতকালের তু...,"['গতকালের', 'খা', 'ভালোই', 'লাগছিলো', 'অর্ডার'...","[গতকাল, খা, ভালো, লাগছিলো, অর্ড, করি, গতকাল, ত...",17,"[NC, NP, NP, VM, NC, VM, NC, NC, JJ, VM, NC, N..."
209,1.0,মালা চপ টাকা একটু হলদে রঙ মিষ্ পরিমাণ স্ একটু ...,মালাই চপ টাকা একটু হলদে রঙের মিষ্টির পরিমাণ স্...,"['মালাই', 'চপ', 'টাকা', 'একটু', 'হলদে', 'রঙের'...","[মালা, চপ, টাকা, একটু, হলদে, রঙ, মিষ্, পরিমাণ,...",15,"[NC, NC, NC, JQ, VM, NC, NP, NC, NC, JQ, NC, N..."
354,2.0,ওরশ আখনি বিরিয়ানির খাবারের মান খুব ভালো পরিমা...,ওরশ আখনি বিরিয়ানির খাবারের মান খুবই ভালো পরিম...,"['ওরশ', 'আখনি', 'বিরিয়ানির', 'খাবারের', 'মান'...","[ওরশ, আখনি, বিরিয়ানির, খাবারের, মান, খুব, ভাল...",12,"[NP, NC, NC, NC, NC, JQ, JJ, NC, JJ, NC, JJ, VM]"
1793,2.0,কোয়ান্ কম এছাড়া জোছ,কোয়ান্টি কম এছাড়া জোছ,"['কোয়ান্টি', 'কম', 'এছাড়া', 'জোছ']","[কোয়ান্, কম, এছাড়া, জোছ]",4,"[NC, JJ, NP, NP]"
2137,0.0,ভাল সার্ভ সার্ভ আগ মত না,ভাল সার্ভিস সার্ভিস আগের মত না,"['ভাল', 'সার্ভিস', 'সার্ভিস', 'আগের', 'মত', 'না']","[ভাল, সার্ভ, সার্ভ, আগ, মত, না]",6,"[JJ, NP, NP, NST, CX, CX]"
1194,2.0,সমুচা খেত মুটামু,সমুচা খেতে মুটামুটি,"['সমুচা', 'খেতে', 'মুটামুটি']","[সমুচা, খেত, মুটামু]",3,"[NC, VM, NC]"
330,1.0,ভালো ত খেয়েছি বাসা একটু দূরে,ভালো ত খেয়েছি বাসা একটু দূরে,"['ভালো', 'ত', 'খেয়েছি', 'বাসা', 'একটু', 'দূরে']","[ভালো, ত, খেয়েছি, বাসা, একটু, দূরে]",6,"[JJ, JJ, VM, NC, JQ, NST]"


In [41]:
df.shape

(2187, 7)

In [42]:
from collections import Counter
from functools import reduce

def find_no_class(count, class_name = ""):
    total = 0
    for key in count.keys():
      if key.startswith(class_name):
        total += count[key]        
    return total


def get_classes(row, grammatical_class = ""):
    count = Counter(row["postags_list"])
    try:
      return find_no_class(count, class_name = grammatical_class)/len(row["postags_list"])
    except ZeroDivisionError:
      return find_no_class(count, class_name = grammatical_class)
    

df["freqAdverbs"] = df.apply(lambda row: get_classes(row, "AMN"), axis = 1)
df["freqPreposition"] = df.apply(lambda row: get_classes(row, "PP"), axis = 1)
df["freqPronoun"] = df.apply(lambda row: get_classes(row, "PPR"), axis = 1)
df["freqVerbs"] = df.apply(lambda row: get_classes(row, "VM"), axis = 1)
df["freqAdjectives"] = df.apply(lambda row: get_classes(row, "JJ"), axis = 1)
df["freqNouns"] = df.apply(lambda row: get_classes(row, ("NC")), axis = 1)
df["freqEnglish"] = df.apply(lambda row: get_classes(row, ("RDF")), axis = 1)

In [43]:
df.head()

,Label,text,clean_text,tokenized_clean_text,tokenized_stem_text,token_length,postags_list,freqAdverbs,freqPreposition,freqPronoun,freqVerbs,freqAdjectives,freqNouns,freqEnglish
0,2.0,পায়েস একটু বাড়ালে আরো ভালো হত,পায়েস একটু বাড়ালে আরো ভালো হতো,"['পায়েস', 'একটু', 'বাড়ালে', 'আরো', 'ভালো', 'হতো']","[পায়েস, একটু, বাড়ালে, আরো, ভালো, হত]",6,"[NP, JQ, NC, JQ, JJ, VM]",0.0,0.0,0.0,0.166667,0.166667,0.166667,0.0
1,1.0,ফ্রেশ খা দুপুরে খেয়ে মজা পেয়ে,ফ্রেশ খা দুপুরে খেয়ে মজা পেয়েছি,"['ফ্রেশ', 'খা', 'দুপুরে', 'খেয়ে', 'মজা', 'পেয...","[ফ্রেশ, খা, দুপুরে, খেয়ে, মজা, পেয়ে]",6,"[NC, NP, NC, VM, NC, VM]",0.0,0.0,0.0,0.333333,0.000000,0.500000,0.0
2,1.0,ঝোলে মাখা ঝরঝরে খিচুড়ি ব্রেডক্রাম্বে মোড়ানো আল...,ঝোলে মাখা ঝরঝরে খিচুড়ি ব্রেডক্রাম্বে মোড়ানো আল...,"['ঝোলে', 'মাখা', 'ঝরঝরে', 'খিচুড়ি', 'ব্রেডক্রা...","[ঝোলে, মাখা, ঝরঝরে, খিচুড়ি, ব্রেডক্রাম্বে, মোড়...",13,"[NC, NC, NC, NC, NC, NP, NP, NC, NC, NP, NP, J...",0.0,0.0,0.0,0.076923,0.000000,0.538462,0.0
3,2.0,কালা ভূনা খেত মুটামু,কালা ভূনা খেতে মুটামুটি,"['কালা', 'ভূনা', 'খেতে', 'মুটামুটি']","[কালা, ভূনা, খেত, মুটামু]",4,"[NP, NC, VM, NC]",0.0,0.0,0.0,0.250000,0.000000,0.500000,0.0
4,0.0,ফুসকা চুপসানো ছিল,ফুসকা চুপসানো ছিল,"['ফুসকা', 'চুপসানো', 'ছিল']","[ফুসকা, চুপসানো, ছিল]",3,"[NC, NV, VM]",0.0,0.0,0.0,0.333333,0.000000,0.333333,0.0


In [44]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/w_post_tag_bangla_speech_tokenized.csv', index = False, header=True)

In [45]:
print("shape of data",df.shape)
print("Number of rows: "+str(df.shape[0]))
print("Number of columns: "+str(df.shape[1]))

shape of data (2187, 14)
Number of rows: 2187
Number of columns: 14


In [47]:
#parcentagewise null values
def missing_data_table(data):
    total = data.isnull().sum().sort_values(ascending=False)
    percent = (data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    return missing_data
missing_data_table(df)

,Total,Percent
Label,0,0.0
text,0,0.0
clean_text,0,0.0
tokenized_clean_text,0,0.0
tokenized_stem_text,0,0.0
token_length,0,0.0
postags_list,0,0.0
freqAdverbs,0,0.0
freqPreposition,0,0.0
freqPronoun,0,0.0
